# ETL Project
### Extract, Transform and Load videogames data

In [2]:
## Dependencies
import pandas as pd

## Console Videogames (data.world)

Dataset retrieved from: https://data.world/sumitrock/videogames

In [6]:
console_df = pd.read_csv('Video_Games.csv')
console_df.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
console_columns = ["Name", "Platform", "Year_of_Release", "Genre", "Developer", "Publisher", "User_Score"]
console_df[console_columns]

,Name,Platform,Year_of_Release,Genre,Developer,Publisher,User_Score
0,Wii Sports,Wii,2006.0,Sports,Nintendo,Nintendo,8
1,Super Mario Bros.,NES,1985.0,Platform,NaN,Nintendo,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,Nintendo,8.3
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,Nintendo,8
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,NaN,Nintendo,NaN
...,...,...,...,...,...,...,...
16714,Samurai Warriors: Sanada Maru,PS3,2016.0,Action,NaN,Tecmo Koei,NaN
16715,LMA Manager 2007,X360,2006.0,Sports,NaN,Codemasters,NaN
16716,Haitaka no Psychedelica,PSV,2016.0,Adventure,NaN,Idea Factory,NaN
16717,Spirits & Spells,GBA,2003.0,Platform,NaN,Wanadoo,NaN


## Steam Videogames (kaggle.com)
Dataset retrieved from: https://www.kaggle.com/nikdavis/steam-store-games

In [ ]:
steam_df = pd.read_csv('steam.csv')

In [16]:
steam_df.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [23]:
steam_columns = ["name", "platforms", "release_date", "genres", "developer", "publisher", "positive_ratings", "negative_ratings"]
b = steam_df[steam_columns]
b

,name,platforms,release_date,genres,developer,publisher,positive_ratings,negative_ratings
0,Counter-Strike,windows;mac;linux,2000-11-01,Action,Valve,Valve,124534,3339
1,Team Fortress Classic,windows;mac;linux,1999-04-01,Action,Valve,Valve,3318,633
2,Day of Defeat,windows;mac;linux,2003-05-01,Action,Valve,Valve,3416,398
3,Deathmatch Classic,windows;mac;linux,2001-06-01,Action,Valve,Valve,1273,267
4,Half-Life: Opposing Force,windows;mac;linux,1999-11-01,Action,Gearbox Software,Valve,5250,288
...,...,...,...,...,...,...,...,...
27070,Room of Pandora,windows,2019-04-24,Adventure;Casual;Indie,SHEN JIAWEI,SHEN JIAWEI,3,0
27071,Cyber Gun,windows,2019-04-23,Action;Adventure;Indie,Semyon Maximov,BekkerDev Studio,8,1
27072,Super Star Blast,windows,2019-04-24,Action;Casual;Indie,EntwicklerX,EntwicklerX,0,1
27073,New Yankee 7: Deer Hunters,windows;mac,2019-04-17,Adventure;Casual;Indie,Yustas Game Studio,Alawar Entertainment,2,0


In [46]:
genre_list = []
for index, row in b.iterrows():
    genre_list.append(row['genres'].split(';')[0])

In [47]:
pd.DataFrame(genre_list)[0].value_counts(ascending=False)

Action                   11212
Adventure                 5256
Casual                    4373
Indie                     2623
Violent                    709
Simulation                 631
Strategy                   532
RPG                        407
Free to Play               395
Sexual Content             245
Racing                     197
Nudity                     112
Sports                      83
Gore                        81
Animation & Modeling        66
Utilities                   60
Design & Illustration       28
Massively Multiplayer       16
Audio Production            16
Education                   14
Early Access                 9
Video Production             5
Accounting                   2
Photo Editing                1
Web Publishing               1
Software Training            1
Name: 0, dtype: int64

In [42]:
console_df['Genre'].value_counts(ascending=False)

Action          3370
Sports          2348
Misc            1750
Role-Playing    1500
Shooter         1323
Adventure       1303
Racing          1249
Platform         888
Simulation       874
Fighting         849
Strategy         683
Puzzle           580
Name: Genre, dtype: int64